### Imports

In [1]:
import numpy as np
import pandas as pd

### Rule-based risk scoring function

In [2]:
def rule_based_score(case: dict) -> float:
    score = 0

    # Duplicate identity risk
    if case.get("registrations_per_aadhaar", 0) >= 3:
        score += 0.25

    # Shared financial identifiers
    if case.get("bank_shared_count", 0) >= 3:
        score += 0.20

    if case.get("phone_shared_count", 0) >= 3:
        score += 0.20

    # Large agent cluster
    if case.get("agent_cluster_size", 0) >= 10:
        score += 0.15

    # Suspiciously low income (example welfare threshold logic)
    income = case.get("annual_income", "0").replace("₹", "").replace(",", "")
    try:
        if int(income) < 50000:
            score += 0.10
    except:
        pass

    return min(score, 1.0)


### Simple ML-Style scoring (simulated model)

In [3]:
def ml_risk_score(case: dict) -> float:
    features = np.array([
        case.get("registrations_per_aadhaar", 0),
        case.get("bank_shared_count", 0),
        case.get("phone_shared_count", 0),
        case.get("agent_cluster_size", 0),
    ], dtype=float)

    # Simple weighted linear model (placeholder for real ML)
    weights = np.array([0.30, 0.25, 0.25, 0.20])

    score = np.dot(features, weights) / (features.max() + 1e-6)

    return float(min(score, 1.0))


### Graph/network risk placeholder

In [4]:
def graph_risk_score(case: dict) -> float:
    phone_degree = case.get("phone_shared_count", 0)
    bank_degree = case.get("bank_shared_count", 0)

    network_score = (phone_degree + bank_degree) / 10
    return float(min(network_score, 1.0))

### Final combined risk score

In [5]:
def final_risk_score(case: dict) -> dict:
    rule_score = rule_based_score(case)
    ml_score = ml_risk_score(case)
    graph_score = graph_risk_score(case)

    # Weighted fusion (research-style hybrid scoring)
    final_score = (
        0.4 * rule_score +
        0.35 * ml_score +
        0.25 * graph_score
    )

    # Risk category
    if final_score >= 0.75:
        level = "HIGH"
    elif final_score >= 0.45:
        level = "MEDIUM"
    else:
        level = "LOW"

    return {
        "rule_score": round(rule_score, 3),
        "ml_score": round(ml_score, 3),
        "graph_score": round(graph_score, 3),
        "final_score": round(final_score, 3),
        "risk_level": level,
    }

### Test with your suspicious case

In [6]:
suspicious_case = {
    "aadhaar_like_id": "4587 9214 6632",
    "phone_number": "9876543210",
    "bank_account": "123456789012",
    "registrations_per_aadhaar": 3,
    "bank_shared_count": 5,
    "phone_shared_count": 4,
    "agent_cluster_size": 18,
    "annual_income": "₹85,000",
    "district": "Bhopal"
}

risk_output = final_risk_score(suspicious_case)

print("Risk Scoring Output:\n")
for k, v in risk_output.items():
    print(f"{k}: {v}")


Risk Scoring Output:

rule_score: 0.8
ml_score: 0.375
graph_score: 0.9
final_score: 0.676
risk_level: MEDIUM
